In [2]:
folder = '/Users/muneebalam/Desktop/Imperial/ML/project/speeches/'
def parse_counterterror():
    fname = 'counterterrorism cfr.txt'
    r = open(folder + fname, 'r')
    data = r.read()
    r.close()
    
    paras = []
    
    data = data.split('ZAKARIA')
    for i in range(len(data)):
        if 'CLINTON' in data[i]:
            data[i] = data[i][data[i].index('CLINTON')+8:]
        else:
            data[i] = ''
        data[i] = [x.strip() for x in data[i].split('\n')]
        for j in range(len(data[i])):
            if len(data[i][j]) > 0:
                paras.append(data[i][j])
    return paras
def parse_me_atl():
    fname = 'middle east atlantic.txt'
    r = open(folder + fname, 'r')
    data = r.read()
    r.close()
    
    data = data.split('JG:')
    paras = []
    for i in range(len(data)):
        if i == 0:
            if 'HILLARY RODHAM CLINTON:' in data[i]:
                data[i] = data[i][data[i].index('HILLARY RODHAM CLINTON:')+23:]
        else:
            if 'HRC:' in data[i]:
                data[i] = data[i][data[i].index('HRC:')+4:]
            else:
                data[i] = ''
        data[i] = [x.strip() for x in data[i].split('\n')]
        for j in range(len(data[i])):
            if len(data[i][j]) > 0:
                paras.append(data[i][j])
    
    return paras
def parse_strint_cfr():
    fname = 'strategic interests cfr.txt'
    r = open(folder + fname, 'r')
    data = r.read()
    r.close()
    
    data = data.split('HAASS:')
    paras = []
    for i in range(len(data)):
        if 'CLINTON:' in data[i]:
            data[i] = data[i][data[i].index('CLINTON:')+8:]
        else:
            data[i] = ''
        data[i] = [x.strip() for x in data[i].split('\n')]
        for j in range(len(data[i])):
            if len(data[i][j]) > 0:
                paras.append(data[i][j])
                
    return paras
def parse_tradepbs():
    fname = 'trade pbs.txt'
    r = open(folder + fname, 'r')
    data = r.read()
    r.close()
    
    data = data.split('JUDY WOODRUFF:')
    paras = []
    for i in range(len(data)):
        if 'HILLARY CLINTON:' in data[i]:
            data[i] = data[i][data[i].index('HILLARY CLINTON:')+16:]
        else:
            data[i] = ''
        data[i] = [x.strip() for x in data[i].split('\n')]
        for j in range(len(data[i])):
            if len(data[i][j]) > 0:
                paras.append(data[i][j])
    return paras

In [3]:
cfrct = parse_counterterror()
atlme = parse_me_atl()
cfrsi = parse_strint_cfr()
pbstr = parse_tradepbs()

In [4]:
combined = cfrct + atlme + cfrsi + pbstr
sources = ['CFR Counterterrorism'] * len(cfrct) + ['The Atlantic']*len(atlme) + ['CFR Middle East']*len(cfrsi) + ['PBS']*len(pbstr)
import pandas as pd
df = pd.DataFrame([tup for tup in zip(sources, combined)], columns=['Source', 'Text'])

In [5]:
def read_nrc():
    #returns {word: (positive/negative, {emotions})}
    r = open('/Users/muneebalam/Desktop/Imperial/ML/project/NRC-Emotion-Lexicon/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt')
    data = r.read().strip()
    r.close()
    
    firstword = 'aback'
    data = data[data.index(firstword):].split('\n')
    lex = {}
    generalset = {'positive', 'negative'}
    for line in data:
        word, emot, posneg = line.split('\t')
        posneg = int(posneg)
        if word not in lex:
            lex[word] = [None, set()]
        if posneg == 1:
            if emot in generalset:
                lex[word][0] = emot
            else:
                lex[word][1].add(emot)
    return lex
sent_dct = read_nrc()

In [6]:
from nltk.corpus import stopwords
stop_word_lst = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
opposites = {'anger': 'fear', 'anticipation': 'surprise', 'disgust': 'trust', 'fear': 'anger', 
             'joy': 'sadness', 'sadness': 'joy', 'surprise': 'anticipation', 'trust': 'disgust',
            'positive': 'negative', 'negative': 'positive'}
negation_words = {'not'}
def split_by_sentences(text):
    return text.replace('?', '.').replace('!', '.').replace(';', '.').split('. ')
def split_by_words(text):
    return [x.strip() for x in text.replace(',', '').replace('-', ' ')
            .replace(':', '').replace('—', '').split(' ')]
def findemot(text):
    emots = {'anger': 0, 'anticipation': 0, 'disgust': 0, 'fear': 0,
        'joy': 0, 'sadness': 0, 'surprise': 0, 'trust': 0}
    posnegneu = {'positive': 0, 'negative': 0, 'neutral': 0}
    wordcount = 0
    #https://en.wikipedia.org/wiki/Contrasting_and_categorization_of_emotions#Plutchik.27s_wheel_of_emotions
    #emotions have opposites
    try:
        sentences = split_by_sentences(text)
        if len(sentences) == 1:
            negation = False
            words = split_by_words(text)
            for word in words:
                wordcount += 1
                if word in negation_words:
                    negation = not negation
                if word in stop_word_lst:
                    pass
                else:
                    if word in sent_dct:
                        if sent_dct[word][0] is None:
                            posnegneu['neutral'] += 1
                        else:
                            if negation:
                                posnegneu[opposites[sent_dct[word][0]]] += 1
                            else:
                                posnegneu[sent_dct[word][0]] += 1
                        for em in sent_dct[word][1]:
                            if negation:
                                emots[opposites[em]] += 1
                            else:
                                emots[em] += 1
                    else:
                        word2 = SnowballStemmer("english").stem(word)
                        if word2 in sent_dct:
                            if sent_dct[word2][0] is None:
                                posnegneu['neutral'] += 1
                            else:
                                if negation:
                                    posnegneu[opposites[sent_dct[word2][0]]] += 1
                                else:
                                    posnegneu[sent_dct[word2][0]] += 1
                            for em in sent_dct[word2][1]:
                                if negation:
                                    emots[opposites[em]] += 1
                                else:
                                    emots[em] += 1
        else:
            sents = [findemot(sen) for sen in sentences]
            for sen_pnn, sen_emot, wc in sents:
                wordcount += wc
                for sen in sen_pnn:
                    posnegneu[sen] += sen_pnn[sen]
                for sen in sen_emot:
                    emots[sen] += sen_emot[sen]
    except AttributeError:
        pass
    return posnegneu, emots, wordcount
def extract_anger(dcts):
    return dcts[1]['anger']
def extract_anticipation(dcts):
    return dcts[1]['anticipation']
def extract_disgust(dcts):
    return dcts[1]['disgust']
def extract_fear(dcts):
    return dcts[1]['fear']
def extract_joy(dcts):
    return dcts[1]['joy']
def extract_sadness(dcts):
    return dcts[1]['sadness']
def extract_surprise(dcts):
    return dcts[1]['surprise']
def extract_trust(dcts):
    return dcts[1]['trust']
def extract_positive(dcts):
    return dcts[0]['positive']
def extract_negative(dcts):
    return dcts[0]['negative']
def extract_neutral(dcts):
    return dcts[0]['neutral']
def extract_wordcount(dcts):
    return dcts[2]

In [7]:
emots = {'anger': extract_anger, 'anticipation': extract_anticipation, 
         'disgust': extract_disgust, 'fear': extract_fear,
        'joy': extract_joy, 'sadness': extract_sadness, 
         'surprise': extract_surprise, 'trust': extract_trust}
posnegneu = {'positive': extract_positive, 'negative': extract_negative, 
             'neutral': extract_neutral}
df['sentinfo'] = df['Text'].apply(findemot) 
for e in emots:
    df[e] = df['sentinfo'].apply(emots[e])
for e in posnegneu:
    df[e] = df['sentinfo'].apply(posnegneu[e])
df['wordcount'] = df['sentinfo'].apply(extract_wordcount)

In [8]:
df.drop('sentinfo', axis=1, inplace=True)
df.to_csv(folder + 'speeches.csv')